Решение нескольких задач для портфолио.\
Задания взяты из реальных примеров с собеседований.\
Поскольку у меня нет оригинальной базы, пришлось брать разные файлы, найденные в открытом доступе.\
Постараюсь к каждому примеров давать аналог кода на SQL.\
Возможно буду дополнять новыми примерами.\
Долистайте до конца, в начале всегда криво, но я специально не убираю такие решения.\

1. Задание.\
   Вывести количество заказов, для тех, кто сделал более 10.\
   
Проверим дата сет на null (есть только в borough).\
На самом деле это проблема, ибо колонка категориальная, как её восполнять - непонятно, так что сделаем вид, что ничего не видели)\

Я могу предположить, что данные можно было бы заполнить, построив распределение по штатам, дням недели и времени суток.\
Оценить данные, чтобы понять, какой показатель лучше подходит: среднее или медиана.\
Но такая схема не учитывает форс-мажоры: погоду, ремонт дорог, праздники, забастовки, эпидемии, сбои в софте такси...\
И это только то, что мне первым пришло в голову, как человеку не имеющему ревалантного обпыта работы в таких компаниях...\

Вернёмся к дата сету.\  
И тут вылезает проблема масштабов, ибо в датасете нет регионов, в которых было бы менее 100 записей...\
Условие на неравенство нужно потому, что иначе количество записей у всех было одинаково.\
Хорошо, изменим уловие, на соотвествующее масштабу...\
Сгруппируем, уберём мультииндекс, превратим в датафрейм, и переименуем результат.

In [16]:
import pandas as pd
import datetime as dt

In [17]:
df = pd.read_csv(r'C:\Users\User\Downloads\2_taxi_nyc.csv', parse_dates=['pickup_dt'])
(df[df['pickups'] != 0]
                      ['borough']
                      .value_counts()
                      .to_frame()
                      .reset_index())
#если добавить такой же фильтр снизу - будет тот же результат, но я решила оставить вывод, чтобы было понято, что отфильтровалось

,borough,count
0,Brooklyn,4335
1,Manhattan,4335
2,Queens,4335
3,Bronx,4334
4,Staten Island,3052
5,EWR,100


In [18]:
#альтернативный вариант
over_10 = (df[df['pickups'] != 0]
           .groupby('borough', as_index=False)[['pickups']]
           .count()
           .rename(columns={'pickups': 'cnt_pickups'})
           .query('cnt_pickups>3052'))
over_10

,borough,cnt_pickups
0,Bronx,4334
1,Brooklyn,4335
3,Manhattan,4335
4,Queens,4335


В MySQL выглядело бы так:\
SELECT client_id, COUNT(order_id) AS over_10\
FROM base\
GROUP BY client_id\
HAVING COUNT(order_id)>10 - на всякий, в некоторых базах вроде не работает алиас в условии...

2. Задание.\
   Средний размер заказа для каждого пользователя за последний месяц.\
   
Тут возникает вопрос: это разовый запрос, или повторяемый?\
Если повторяемый, пришлось бы приязывать к NOW() или datime.now()\
Так же непонятно, что автор задания понимает продажи за последний месяц?\
Все продажи с начала месяца? А, если сейчас 5 число?)\
Так что предположим, что:\
    - База постоянно пополняется, т.е это не архив.\
    - Мы подразумеваем буквально период - последние 30 дней, и нам чётко известны эти рамки.\
Поскольку у меня как раз-таки статичный снимок базы, для начала выясним последнюю дату в ней:

In [19]:
#df.iloc[df['pickup_dt'].idxmax(), 0] - я тут не много перемудрила, зато вспомнила полезнуюю функцию)
df['pickup_dt'].max()

Timestamp('2015-06-30 23:00:00')

In [20]:
avg_last_month = (df[df['pickup_dt'].between('2015-05-30 23:00:00', '2015-06-30 23:00:00')]
                   .groupby('borough', as_index=False)[['pickups']]
                   .mean()
                   .rename(columns={'pickups': 'mean'}))
avg_last_month

,borough,mean
0,Bronx,68.543624
1,Brooklyn,678.595973
2,EWR,0.038926
3,Manhattan,2771.889933
4,Queens,399.846980
5,Staten Island,2.335570


В MySQL выглядело бы так:\
SELECT client_id, AVG(order_id)AS avg_last_month\
FROM base\
WHERE order_date BETWEEN '2015-05-30' AND '2015-06-30'

3. Задание.\
   Для топ 10 пользователей с максимальным количеством заказов за последний год, вычислить размер среднего заказа за последний месяц.\
   
С одной стороны - у нас уже есть 2 таблица со средними продажами за последний месяц.\
Теперь нам нужно только её правильно отфильтровать.\
Но тут встаёт вопрос:\
     - 10 это ранг или количество?\
     - Если ранг, то как его считать?\
Но для простоты, предположим опять, что нам просто нужен верх списка.\
Тем не менее в моей базе нет даже 10 уникальных значений, значит, будем выводить 5)

In [21]:
#over_10[over_10['pickup_dt'].between('2014-06-30 23:00:00', '2015-06-30 23:00:00')]
#  тут я вспомнила, что в этом снимке данные примерно за полгода, значит ищем минимум.
df.iloc[df['pickup_dt'].idxmin(), 0]

Timestamp('2015-01-01 01:00:00')

In [22]:
top_5 = (df[df['pickup_dt'].between('2015-03-30 23:00:00', '2015-06-30 23:00:00') 
            & (df['pickups'] != 0)]['borough']
           .value_counts()
           .to_frame()
           .reset_index()
        ).head(5)
top_5

,borough,count
0,Bronx,2209
1,Brooklyn,2209
2,Manhattan,2209
3,Queens,2209
4,Staten Island,1684


In [23]:
avg_last_month_5 = pd.merge(left=top_5
                            , right=avg_last_month
                            , how='left'
                            , on='borough')
avg_last_month_5.drop('count', axis=1)

,borough,mean
0,Bronx,68.543624
1,Brooklyn,678.595973
2,Manhattan,2771.889933
3,Queens,399.846980
4,Staten Island,2.335570


В MySQL выглядело бы так (у меня поехало всё оформление, пришлось импровизировать...):\

WITH CTE AS(\
............................SELECT client_id, COUNT(order_id) AS top_10\
............................# или COUNT(DISTINCT(order_id)), если в этой форме есть разбивка по продуктам в заказе.\
............................FROM base\
............................WHERE order_date BETWEEN '2014-06-30' AND '2015-06-30'\
............................GROUP BY client_id\
............................ORDER BY COUNT(order_id) DESC - здесь аналогично возможно нужен DISTINCT\
............................LIMIT 10\
.........................)\

SELECT b.client_id, AVG(b.order_id)AS avg_last_month\
FROM base as b\
..............RIGHT JOIN CTE ON(client_id)\
WHERE order_date BETWEEN '2015-03-30' AND '2015-06-30'     

In [24]:
#Альтернативные решения
fltr = top_5['borough'].to_list()
fltr

['Bronx', 'Brooklyn', 'Manhattan', 'Queens', 'Staten Island']

In [25]:
avg_last_v_2 = avg_last_month[avg_last_month['borough'].isin(fltr)]
avg_last_v_2

,borough,mean
0,Bronx,68.543624
1,Brooklyn,678.595973
3,Manhattan,2771.889933
4,Queens,399.846980
5,Staten Island,2.335570


В SQL не вижу смысла делать через IN, это более медленный вариант, чем JOIN, можно было бы через WHERE EXISTS.\
Предлагаю посмотреть код с той же структурой, но для условия ранг топ-10(сойдёмся на ранге без пропусков).\
Тут не стала мудрить с точками, оформление тоже самое...\
Группировка в окне имеет смысл, если эта таблица имеет повторы по order_id\
\
WITH CTE AS( SELECT client_id, order_id\
            , DENSE_RUNC OVER(\
                                      PARTITION BY client_id\
                                      ORDER BY COUNT(DISTINC(order_id)) DESC) AS over_10\
             FROM base\
             WHERE order_date BETWEEN '2014-06-30' AND '2015-06-30'\
             GROUP BY client_id, order_id\
)\

SELECT b.client_id, AVG(b.order_id)AS avg_last_month\
FROM base as b\
     RIGHT JOIN CTE ON(client_id)\
WHERE order_date BETWEEN '2015-03-30' AND '2015-06-30'
  AND over_10 <= 10

 4. Задание.\
   Посчитать среднее за каждый месяц в текущем году и сравнить его\
   с показателем за соотвествующий месяц в прошлом году.\

   Не понятно, что имел в виду автор: абсолютное или процентное значение, посчитаеми и то, и то.\
   Конечно, самое простое решение - добавить две колонки с фильтрами по дате, затем вычесть одну из другой...\
   Но, как правило, в таких заданиях требуют не добавлять лишних столбцов, не использовать подзапросы...\
   Ещё вопрос в том, какое значение считать опорным - этот, или прошлый год?\
   Опять таки же встаёт вопрос: должна ли дата вычислятся динамически...\
   В общем, будем за 100% считать прошлый год, и то, что нам опять  известны точные границы периодов.\
   Предыдущий датасет не подходит - в нём данные только за полгода, возьму другой, с курсов со степик.\

In [26]:
df_1=pd.read_csv(r'C:\Users\User\Downloads\1.5.1.csv', parse_dates=['Date'])
df_1

,Date,Region,Product,SalesCount
0,2022-01-03,South,Product C,716
1,2022-01-04,South,Product C,1623
2,2022-01-07,South,Product A,1159
3,2022-01-12,West,Product B,501
4,2022-01-21,East,Product B,1567
...,...,...,...,...
205,2023-12-02,North,Product A,1288
206,2023-12-19,North,Product B,798
207,2023-12-23,North,Product B,1145
208,2023-12-26,South,Product B,1072


В итоге, самым простым мне показалось вернуть сводную + 2 вычисляемых столбца.\
Но это было далеко не первое решение, да и сам этот скрипт я правила не раз...\
Но честно сама пришла, искала только как работают отдельные методы.

In [27]:
avg_diff_per_month_2 = (df_1
                       .pivot_table(values='SalesCount', 
                                    index=df_1.Date.dt.month, 
                                    columns=df_1.Date.dt.year)
                        )
avg_diff_per_month_2['Abs_diff'] = avg_diff_per_month_2[2023] - avg_diff_per_month_2[2022]
avg_diff_per_month_2['Prc_diff'] = avg_diff_per_month_2[2023] * 100 / avg_diff_per_month_2[2022]
avg_diff_per_month_2.reset_index(drop=True)

Date,2022,2023,Abs_diff,Prc_diff
0,1061.333333,1368.625000,307.291667,128.953361
1,1202.875000,1226.444444,23.569444,101.959426
2,1263.571429,1248.700000,-14.871429,98.823064
3,1547.000000,1456.090909,-90.909091,94.123524
4,1151.833333,1240.714286,88.880952,107.716477
5,1382.555556,1240.615385,-141.940171,89.733492
6,1223.285714,1245.181818,21.896104,101.789942
7,1287.428571,1378.076923,90.648352,107.041039
8,1044.000000,1210.000000,166.000000,115.900383
9,1436.428571,1216.142857,-220.285714,84.664346


В SQL первое очевидное решение - создать примерно тот же код, что и снизу, в неудачных решениях:\
Сделать две CTE за этот и прошлый год с группировкой по месяцам, рассчитать среднее,\
сделать в каждой дополнительный столбец с месяцем, как id записи.
Далее просто INNER JOIN по месяцу и из него в SELECT два вычсисляемых столбца.\

Второе, что пришло мне в голову - это оконка. Но в этом решении я до конца не уверена.\
Примерно должно выглядеть так:\
- В SELECT среднее и оконка.\ 
- Группируем и сортируем по году, затем месяцу, вычисляем среднее.\
- Поитогу получаем таблицу, где под январём того года, январь текущего.\
- Вот тут тонкий момент: я проверяла на таблице только за 1 год, не до конца уверена, что в окне не нужна партиция.\
- Теперь делаем вычисляемый столбец: среднее - LAG(среднее) в партиции(на моих данных это дало null)\
  и сортировке дублируем всё, как внутри запроса.\
- Процент вычисляем аналогично...
- Теперь, при желании, можно завернуть результат в CTE, отфильтровать из него прошлый год, округлить значения, если нужно.


А вот как бы выглядело аналогичное решение в Pandas:

In [28]:
avg_diff_per_month_3 = (df_1
                        .groupby([df_1.Date.dt.year, df_1.Date.dt.month])
                        [['SalesCount']]
                        .mean()
                        .sort_index(level=1)
                        .rename(columns={'SalesCount': 'Avg_2023'})
                        )
#Добавим нужные колонки:
avg_diff_per_month_3['Abs_diff'] = avg_diff_per_month_3['Avg_2023'].diff()
avg_diff_per_month_3['Prc_diff'] = avg_diff_per_month_3['Avg_2023'].pct_change()

#Теперь приведём датасет в порядок:
avg_diff_per_month_3.index.rename(['Year', 'Month'], inplace=True)                     
avg_diff_per_month_3 = avg_diff_per_month_3.reset_index()
                        
avg_diff_per_month_3 = (avg_diff_per_month_3[avg_diff_per_month_3['Year'] == 2023]
                        .drop(['Year', 'Month'], axis=1))
avg_diff_per_month_3

#avg_diff_per_month_3[1::2].reset_index(drop=True) - это было 1 решение, но я довела его до ума.
#При желании можно привести процентный столбец к формату в первом решении. 
#Но я решила оставить так, так как осознала, что возможно нужен был именно такой вариант вычисления.

,Avg_2023,Abs_diff,Prc_diff
1,1368.625000,307.291667,0.289534
3,1226.444444,23.569444,0.019594
5,1248.700000,-14.871429,-0.011769
7,1456.090909,-90.909091,-0.058765
9,1240.714286,88.880952,0.077165
11,1240.615385,-141.940171,-0.102665
13,1245.181818,21.896104,0.017899
15,1378.076923,90.648352,0.070410
17,1210.000000,166.000000,0.159004
19,1216.142857,-220.285714,-0.153357


## Бонусы ##
- Различные приёмы из неудачных решений,
- Разовые рассчёты метрик.

In [37]:
#Кусок нерабочего решения, просто интересный способ работы с датами + метрика
avg_diff_2 = df_1.copy()
avg_diff_2['Date_prev'] = avg_diff_2['Date'] - pd.tseries.offsets.DateOffset(years = 1)
avg_diff_2['Date_m'] = df_1['Date'] + pd.offsets.MonthEnd(0) - pd.offsets.MonthBegin()

#Нагляднее, наверное было бы делать такую метрику с группировкой;
avg_diff_2['Avg_x'] = avg_diff_2['SalesCount'].transform(lambda x: x - x.mean())

#Нашла решение под группировку:
avg_diff_2 = (avg_diff_2
                        .assign(Avg__group = lambda x: x
                               .groupby(['Region', 'Product'])['SalesCount']
                               .transform(lambda x: x - x.mean())))
#Даже такая группировка позволяет увидеть, как меняется отклонение с учётом региона.
avg_diff_2

,Date,Region,Product,SalesCount,Date_prev,Date_m,Avg_x,Avg__group
0,2022-01-03,South,Product C,716,2021-01-03,2022-01-01,-555.342857,-503.562500
1,2022-01-04,South,Product C,1623,2021-01-04,2022-01-01,351.657143,403.437500
2,2022-01-07,South,Product A,1159,2021-01-07,2022-01-01,-112.342857,40.222222
3,2022-01-12,West,Product B,501,2021-01-12,2022-01-01,-770.342857,-773.000000
4,2022-01-21,East,Product B,1567,2021-01-21,2022-01-01,295.657143,322.038462
...,...,...,...,...,...,...,...,...
205,2023-12-02,North,Product A,1288,2022-12-02,2023-12-01,16.657143,-36.000000
206,2023-12-19,North,Product B,798,2022-12-19,2023-12-01,-473.342857,-366.555556
207,2023-12-23,North,Product B,1145,2022-12-23,2023-12-01,-126.342857,-19.555556
208,2023-12-26,South,Product B,1072,2022-12-26,2023-12-01,-199.342857,-126.809524


In [33]:
df_2 = (df_1
            .groupby([df_1.Date.dt.year, df_1.Date.dt.month])
            [['SalesCount']]
            .mean()
            .sort_index(level=1)
            .rename(columns={'SalesCount': 'Avg_2023'})
             )
df_2['Abs_diff'] = df_2['Avg_2023'].diff()
df_2['Prc_diff'] = df_2['Avg_2023'].pct_change()
df_2.swaplevel(0, 1).unstack().reset_index(drop=True)
#Технически решение со срезом в 4 задании верное, но я решила попробовать что-то другое, вышло криво, но тоже работает.

Avg_2023                 Abs_diff              Prc_diff          
Date         2022         2023        2022        2023      2022      2023
0     1061.333333  1368.625000         NaN  307.291667       NaN  0.289534
1     1202.875000  1226.444444 -165.750000   23.569444 -0.121107  0.019594
2     1263.571429  1248.700000   37.126984  -14.871429  0.030272 -0.011769
3     1547.000000  1456.090909  298.300000  -90.909091  0.238888 -0.058765
4     1151.833333  1240.714286 -304.257576   88.880952 -0.208955  0.077165
5     1382.555556  1240.615385  141.841270 -141.940171  0.114322 -0.102665
6     1223.285714  1245.181818  -17.329670   21.896104 -0.013969  0.017899
7     1287.428571  1378.076923   42.246753   90.648352  0.033928  0.070410
8     1044.000000  1210.000000 -334.076923  166.000000 -0.242423  0.159004
9     1436.428571  1216.142857  226.428571 -220.285714  0.187131 -0.153357
10    1170.000000  1284.916667  -46.142857  114.916667 -0.037942  0.098219
11    1351.000000  1091.666667   66.083333 -259.333333  0.051430 -0.191957

In [ ]:
#Первое кривое решение 4 задания.
avg_diff_per_month = (df_1.groupby(df_1[df_1['Date']
                                   .between('2023-01-01', '2023-12-31')]['Date']
                          .dt.to_period("M"))['SalesCount']
                          .mean()
                          .reset_index(drop=True)
                          
                     - df_1.groupby(df_1[df_1['Date']
                                   .between('2022-01-01', '2022-12-31')]['Date']
                          .dt.to_period("M"))['SalesCount']
                          .mean()
                          .reset_index(drop=True)
                     ).to_frame()
avg_diff_per_month